# tweet grab

In [1]:
import os
import tweepy
import logging
import pprint
import json
import time
import boto3
import datetime
import pandas as pd
import numpy as np
print('imported modules successfully.')

imported modules successfully.


### set api creds

In [2]:
base = os.getcwd()
data_dir = os.path.join(base, 'data')
log_dir = os.path.join(base, 'logs')
pp = pprint.PrettyPrinter(indent = 1)

os.chdir(base)

with open('./../creds.json', 'r') as f:
    tweepy_creds = json.load(f)
f.close()
    
auth = tweepy.OAuthHandler(tweepy_creds['twitter-api-key'], tweepy_creds['twitter-secret-key'])
auth.set_access_token(tweepy_creds['twitter-access-token'], tweepy_creds['twitter-secret-access'])

api = tweepy.API(auth)
logging.debug('authorized API w/ client id and secret.')
print('initialized modules.')

initialized modules.


### get followers

In [6]:
def get_followers(user):
    following = []
    friends_response = api.get_friends(screen_name = user, count = 100, cursor = -1)
    following.extend([t.screen_name for t in friends_response[0]])
    next_cursor = friends_response[1][1]
    print("got most recent 100 following")

    while next_cursor != 0:
        try:
            friends_response = api.get_friends(screen_name = user, count = 100, cursor = next_cursor)
            following.extend([t.screen_name for t in friends_response[0]])
            next_cursor = friends_response[1][1]
            print("got next 100 following")
        
        except:
            time.sleep(60*15)

    return(following)

In [7]:
my_following = get_followers("xanax_princess_")

got most recent 100 following
got next 100 following
got next 100 following
got next 100 following


In [8]:
print(len(my_following))

383


### get tweets per user

In [79]:
def get_tweets(user):
    print("scraping tweets for user: {}".format(user))
    i = 0
    tweets = []
    timeline_response = api.user_timeline(screen_name = user, count = 100)
    tweets.extend([(t.user.screen_name, t.text, t.id, t.created_at, t.retweet_count, t.favorite_count) for t in timeline_response])
    n = len(timeline_response)
    last_id = tweets[n-1][1]
    i += 1

    while (i < 20):
        try:
            timeline_response = api.user_timeline(screen_name = user, count = 100, max_id = last_id)
            tweets.extend([(t.user.screen_name, t.text, t.id, t.created_at, t.retweet_count, t.favorite_count) for t in timeline_response])
            n = len(timeline_response)
            last_id = tweets[n-1][1]
            i += 1
        
        except Exception as e:
            print(e)
            if "429" in str(e):
                print("sleeping for 5 minutes")
                time.sleep(60*5)

            else:
                print("sleeping for a minute")
                time.sleep(60)

    return(tweets)

In [61]:
my_tweets = get_tweets(my_following[0])

scraping tweets for user: _nyancrimew


In [63]:
# my_tweets

In [64]:
# timeline_response = api.user_timeline(screen_name = my_following[0], count = 100)
# timeline_response[0]._json

In [65]:
# my_tweets

## get all tweets

In [75]:
subset = my_following[18:40]

In [76]:
master_tweets = []

for f in subset:
    output = get_tweets(f)
    master_tweets.extend(output)
    time.sleep(5)

scraping tweets for user: youngtiddy
scraping tweets for user: FranziaMom
scraping tweets for user: pilatesdev
scraping tweets for user: mike_julian
scraping tweets for user: knifeplay_music
scraping tweets for user: joneaux
scraping tweets for user: GergelyOrosz
scraping tweets for user: p8stie
scraping tweets for user: shanselman
scraping tweets for user: arturo182
scraping tweets for user: chompie1337
scraping tweets for user: OliverBaxxter
scraping tweets for user: nothingnowhere
scraping tweets for user: Senn_Spud
scraping tweets for user: frankivelli
scraping tweets for user: page_eco
scraping tweets for user: adriyoung
scraping tweets for user: wildtiktokss
scraping tweets for user: 23Five
scraping tweets for user: stinkykatie
scraping tweets for user: thisisnaomie
scraping tweets for user: whotfisjovana


In [77]:
df = pd.DataFrame(master_tweets, columns = ['user', 'text', 'id', 'created_at', 'retweet_count', 'favorite_count'])
df.head()

,user,text,id,created_at,retweet_count,favorite_count
0,youngtiddy,@thepoleninja like bitch i’m just eating break...,1616924863645777923,2023-01-21 22:25:03+00:00,0,11
1,youngtiddy,male birth control? thank god. way too many ma...,1616870084907261952,2023-01-21 18:47:22+00:00,32,407
2,youngtiddy,i’d smoke it here and there i’d smoke it anywhere,1616860690073128962,2023-01-21 18:10:02+00:00,18,348
3,youngtiddy,@KMRTMoose !!!,1616849585498120195,2023-01-21 17:25:55+00:00,0,9
4,youngtiddy,@redpyrameadhead acting like i’m responsible l...,1616849452328947713,2023-01-21 17:25:23+00:00,0,26


In [78]:
df.tail()

,user,text,id,created_at,retweet_count,favorite_count
2079,whotfisjovana,"@darlingchandIer literally I’d be hugging, and...",1616666452525719552,2023-01-21 05:18:13+00:00,2,13
2080,whotfisjovana,i mean i loveee myself but in a more romantic ...,1616665986026901504,2023-01-21 05:16:21+00:00,2,60
2081,whotfisjovana,i wish i fell in love with someone like myself...,1616665713585823745,2023-01-21 05:15:16+00:00,422,2954
2082,whotfisjovana,what makes my beautiful heart look so stompable,1616665289998876673,2023-01-21 05:13:35+00:00,35,180
2083,whotfisjovana,truly reminds me that the only person who has ...,1616663105290878976,2023-01-21 05:04:55+00:00,7,68


In [81]:
master_tweets_df = pd.concat([master_tweets_df, df], axis = 0).reset_index(drop = True)

In [82]:
master_tweets_df.head()

,user,text,id,created_at,retweet_count,favorite_count
0,_nyancrimew,@coolartcorner OMG I LOVE THESE,1617029128032600064,2023-01-22 05:19:21+00:00,0,4
1,_nyancrimew,RT @coolartcorner: holy fucking bingle! https:...,1617029094373478400,2023-01-22 05:19:13+00:00,18,0
2,_nyancrimew,RT @jerl_0: holy fucking bingle. what?! :3,1617028264400142338,2023-01-22 05:15:55+00:00,5,0
3,_nyancrimew,RT @unixanimals: BINGLE-SUPPORTED\nJOIN YHE :3...,1617027753504653312,2023-01-22 05:13:54+00:00,2,0
4,_nyancrimew,@TempleOfTefnut i have a little experience\n\n...,1617025824024301568,2023-01-22 05:06:13+00:00,0,4


In [83]:
master_tweets_df.tail()

,user,text,id,created_at,retweet_count,favorite_count
3745,whotfisjovana,"@darlingchandIer literally I’d be hugging, and...",1616666452525719552,2023-01-21 05:18:13+00:00,2,13
3746,whotfisjovana,i mean i loveee myself but in a more romantic ...,1616665986026901504,2023-01-21 05:16:21+00:00,2,60
3747,whotfisjovana,i wish i fell in love with someone like myself...,1616665713585823745,2023-01-21 05:15:16+00:00,422,2954
3748,whotfisjovana,what makes my beautiful heart look so stompable,1616665289998876673,2023-01-21 05:13:35+00:00,35,180
3749,whotfisjovana,truly reminds me that the only person who has ...,1616663105290878976,2023-01-21 05:04:55+00:00,7,68
